In [ ]:
import os

In [1]:
import pandas as pd
import networkx as nx

In [2]:
from iac_sketch import data, etl, sketch, transform
from iac_sketch.extract import extract_python, extract_yaml
from iac_sketch import system_tests

In [3]:
# DEBUG
import importlib

importlib.reload(extract_yaml)
importlib.reload(extract_python)
importlib.reload(etl)
importlib.reload(data)
importlib.reload(transform)
importlib.reload(system_tests)
importlib.reload(sketch)

<module 'iac_sketch.sketch' from '/Users/zhafen/repos/iac-sketch/iac_sketch/sketch.py'>

In [ ]:
architect = sketch.Architect(root_dir="..")
registry = architect.perform_registry_etl()
tests, test_results = architect.validate_registry(
    test_kwargs={
        "iac_sketch/system_tests.test_designed": {
            "allowed_infrastructure": ["iac_sketch_infrastructure"],
        },
        "iac_sketch/system_tests.test_implemented": {
            "allowed_infrastructure": ["iac_sketch_infrastructure"],
        },
    }
)

entity: iac_sketch/system_tests.test_designed
requirement: fully_designed
priority: 0.9
description:
    Each [requirement] entity has at least one [satisfies] entity or one [child]
    entity.
test_passed: False
failed_components:


,entity,comp_key,priority,value,link.link_type,link.source,link.target,status.comp_key,status.value
1,documents_system_architecture,1,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN
4,facilitates_a_source_of_truth,0,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN
18,records_system_architecture,0,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN
19,validates_system_architecture,1,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN


--------------------------------------------------------------------------------

entity: iac_sketch/system_tests.test_defined
requirement: fully_defined
priority: 0.8
description:
    All [compdef] entities are defined and valid.
test_passed: True
--------------------------------------------------------------------------------

entity: iac_sketch/system_tests.test_implemented
requirement: fully_implemented
priority: 0.7
description:
    Meets the same criteria as "fully_designed" and all [satisfies] entities
    either have a test that passes when executed or have a status of "in
    production".
test_passed: False
failed_components:


,entity,comp_key,priority,value,link.link_type,link.source,link.target,status.comp_key,status.value
0,can_incorporate_poorly_formed_components,1,0.5,iac_sketch_infrastructure,satisfies,test_can_incorporate_poorly_formed_components,can_incorporate_poorly_formed_components,NaN,NaN
1,documents_system_architecture,1,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN
4,facilitates_a_source_of_truth,0,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN
6,fully_connected,1,0.4,NaN,satisfies,iac_sketch/system_tests.test_connected,fully_connected,NaN,NaN
7,fully_defined,1,0.8,NaN,satisfies,iac_sketch/system_tests.test_defined,fully_defined,NaN,NaN
8,fully_designed,1,0.9,NaN,satisfies,iac_sketch/system_tests.test_designed,fully_designed,NaN,NaN
9,fully_implemented,1,0.7,NaN,satisfies,iac_sketch/system_tests.test_implemented,fully_implemented,NaN,NaN
17,no_forbidden_components,1,0.5,NaN,satisfies,iac_sketch/system_tests.test_no_forbidden_comp...,no_forbidden_components,NaN,NaN
18,records_system_architecture,0,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN
19,validates_system_architecture,1,0.5,iac_sketch_infrastructure,NaN,NaN,NaN,NaN,NaN


--------------------------------------------------------------------------------

entity: iac_sketch/system_tests.test_no_forbidden_components
requirement: no_forbidden_components
priority: 0.5
description:
    No [error], [todo], or other "forbidden" components.
test_passed: False
failed_components:


component_type
entity                                             comp_key               
FunctionDef                                        3                  todo
Module                                             2                  todo
child_component                                    1                  todo
iac_sketch/etl.TransformSystem.build_graph_from... 0                  todo
iac_sketch/sketch.Architect.perform_registry_etl   2                  todo
iac_sketch/sketch.Architect.validate_registry      2                  todo
                                                   3                  todo
                                                   4                  todo
iac_sketch/system_tests.test_implemented           2                  todo
iac_sketch/system_tests.test_no_forbidden_compo... 2                  todo
iac_sketch/transform.ComponentValidator            0                  todo
parent                                             4                  todo

--------------------------------------------------------------------------------

entity: iac_sketch/system_tests.test_connected
requirement: fully_connected
priority: 0.4
description:
    The network composed of [link] components and entities as nodes is fully
    connected.
test_passed: False
failed_components:


,connected_component_group,comp_key
entity,,
Call,12,5
ClassDef,13,5
FunctionDef,14,6
Import,15,5
ImportFrom,16,5
...,...,...
test_can_incorporate_poorly_formed_components,2,5
todo,643,5
url,644,5


--------------------------------------------------------------------------------



In [ ]:
# Visualize the graph
nx.draw(registry.graph, with_labels=True)

In [ ]:
import ast

In [ ]:
with open("../iac_sketch/etl.py", "r") as f:
    source = f.read()
tree = ast.parse(source)
tree

In [ ]:
list(ast.iter_child_nodes(tree))

In [ ]:
import ast

In [ ]:
node = ast.parse("""
def my_function(x) -> int:
    return x + 1

x = my_function(5)
""")

In [ ]:
nt = ast.NodeTransformer()
nt.visit(node)

In [ ]:
nt = ast.NodeVisitor()
nt.visit(node)

In [ ]:
list(ast.iter_child_nodes(node.body[1]))

In [ ]:
node.body[1].value.func.ctx

In [ ]:
node.body[0].__class__.__name__

In [ ]:
node.body[0]._fields

In [ ]:
node.body[0].body[0].value._fields

In [ ]:
visitor = ComponentVisitor()

In [ ]:
visitor.visit(tree)

In [ ]:
pd.DataFrame(visitor.components)

In [ ]:
type(tree.body[-1])

In [ ]:
dict(ast.iter_fields(child_node))

In [ ]:
components = []
for child_node in ast.iter_child_nodes(tree.body[-1]):
    comp_i = {
        "component_type": type(child_node).__name__
    }
    components.append(comp_i)

components

In [ ]:
tree.body[-1].__dir__()

In [ ]:
for item in ast.iter_fields(tree.body[-1].body[-1]):
    print(item)

In [ ]:
visitor.visit(tree.body[-1])

In [ ]:
ast.walk(visitor.visit(tree))

In [ ]:
for item in ast.walk(tree):
    print(item)

In [ ]:
visitor.visit_Constant(tree)

In [ ]:
components = {}
for field, value in ast.iter_fields(tree):
    print(field, value)
    if isinstance(value, str)

In [ ]:
for item in ast.iter_child_nodes(transform_sys_tree):
    print(item)

In [ ]:
ast.get_source_segment(source, transform_sys_tree)

In [ ]:
ast.get_docstring(transform_sys_tree)

In [ ]:
isinstance(transform_sys_tree, ast.AST)